In [7]:
import os
import random

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler

from datasets import UserModeDataset
from models import DVBPR
from trainers import Trainer
from utils.data import extract_embedding


# Parameters
RNG_SEED = 0
EMBEDDING_PATH = os.path.join("data", "embedding-resnet50.npy")
TRAINING_PATH = os.path.join("data", "naive-user-train.csv")
VALIDATION_PATH = os.path.join("data", "naive-user-validation.csv")
CHECKPOINTS_DIR = os.path.join("checkpoints")
USE_GPU = True

# Parameters (training)
SETTINGS = {
    "dataloader:batch_size": 42_000,
    "dataloader:num_workers": os.cpu_count(),
    "model:dim_latent": 200,
    "model:dim_visual": 100,
    "optimizer:lr": 0.001,
    "optimizer:weight_decay": 0.0001,
    "scheduler:factor": 0.6,
    "scheduler:patience": 2,
    "train:max_epochs": 1,#5,#150,
    "train:max_lrs": 5,
    "train:non_blocking": True,
    "train:train_per_valid_times": 1,
}


In [9]:
%%time
# Freezing RNG seed if needed
if RNG_SEED is not None:
    print(f"\nUsing random seed...")
    random.seed(RNG_SEED)
    torch.manual_seed(RNG_SEED)
    np.random.seed(RNG_SEED)

# Load embedding from file
print(f"\nLoading embedding from file... ({EMBEDDING_PATH})")
embedding = np.load(EMBEDDING_PATH, allow_pickle=True)

# Extract features and "id2index" mapping
print("\nExtracting data into variables...")
embedding, _, _ = extract_embedding(embedding, verbose=True)
print(f">> Features shape: {embedding.shape}")

# DataLoaders initialization
print("\nInitialize DataLoaders")
# Training DataLoader
train_dataset = UserModeDataset(
    csv_file=TRAINING_PATH,
)
print(f">> Training dataset: {len(train_dataset)}")
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=SETTINGS["dataloader:batch_size"],
    num_workers=SETTINGS["dataloader:num_workers"],
    pin_memory=True,
)
print(f">> Training dataloader: {len(train_dataloader)}")
# Validation DataLoader
valid_dataset = UserModeDataset(
    csv_file=VALIDATION_PATH,
)
print(f">> Validation dataset: {len(valid_dataset)}")
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=SETTINGS["dataloader:batch_size"],
    num_workers=SETTINGS["dataloader:num_workers"],
    pin_memory=True,
)
print(f">> Validation dataloader: {len(valid_dataloader)}")
# Model initialization
print("\nInitialize model")
device = torch.device("cuda:0" if torch.cuda.is_available() and USE_GPU else "cpu")
if torch.cuda.is_available() != USE_GPU:
    print((f"\nNotice: Not using GPU - "
           f"Cuda available ({torch.cuda.is_available()}) "
           f"does not match USE_GPU ({USE_GPU})"
    ))
N_USERS = len(set(train_dataset.ui))
N_ITEMS = len(embedding)
print(f">> N_USERS = {N_USERS} | N_ITEMS = {N_ITEMS}")
print(torch.Tensor(embedding).shape)
model = DVBPR(
    N_USERS, N_ITEMS,  # Number of users and items
    SETTINGS["model:dim_latent"], SETTINGS["model:dim_visual"],  # Size of internal spaces
).to(device)

# Training setup
print("\nSetting up training")
optimizer = optim.Adam(
    model.parameters(),
    lr=SETTINGS["optimizer:lr"],
    weight_decay=SETTINGS["optimizer:weight_decay"],
)
criterion = nn.BCEWithLogitsLoss(reduction="sum")
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", factor=SETTINGS["scheduler:factor"],
    patience=SETTINGS["scheduler:patience"], verbose=True,
)



Using random seed...

Loading embedding from file... (data/embedding-resnet50.npy)

Extracting data into variables...
>> Features shape: (9633, 2048)

Initialize DataLoaders
>> Training dataset: 5000086
>> Training dataloader: 120
>> Validation dataset: 500656
>> Validation dataloader: 12

Initialize model

Notice: Not using GPU - Cuda available (False) does not match USE_GPU (True)
>> N_USERS = 1079 | N_ITEMS = 9633
torch.Size([9633, 2048])

Setting up training
CPU times: user 2.62 s, sys: 296 ms, total: 2.92 s
Wall time: 2.92 s


In [10]:
%%time
# Training
version = f"VBPR_wikimedia"
trainer = Trainer(
    model, device, criterion, optimizer, scheduler,
    checkpoint_dir=CHECKPOINTS_DIR,
    version=version,
)
best_model, best_acc, best_loss, best_epoch = trainer.run(
    SETTINGS["train:max_epochs"], SETTINGS["train:max_lrs"],
    {"train": train_dataloader, "validation": valid_dataloader},
    train_valid_loops=SETTINGS["train:train_per_valid_times"],
)


>> Model version: VBPR_wikimedia
>> Checkpoints stored at... checkpoints/VBPR_wikimedia.tar


RuntimeError: shape '[-1, 3, 224, 224]' is invalid for input of size 42000

In [ ]:
# Final result
print(f"\nBest ACC {best_acc} reached at epoch {best_epoch}")
print(best_model)

In [14]:
train_dataset[2]

(0, 276, 7961)